In [ ]:
import sys
import serial

new_path = '../scripts/'

if new_path not in sys.path:
    sys.path.append(new_path)
    
import target_com as com
import shelve

In [ ]:
if "m4f" not in dir():
    global m4f
    m4f = True # implementation switch, either use m4f or mupq/opt implementation
if "fisher" not in dir():
    global fisher
    fisher = False
if "loop" not in dir():
    global loop
    loop = 1 # values {0,1,2}

if "ff" not in dir():
    global ff
    ff = f"{'' if m4f else 'mupq_'}crypto_kem_bike{lvl}_{'fisher_yates' if fisher else 'set_bits'}_{loop}_{'m4f' if m4f else 'opt'}_fi"

print(ff)

In [ ]:
# load previosly stored faulted keys
with shelve.open("../traces/key_dump") as data:
	key_dump = data[ff]
	# key_dump = data["current"]

Start the firmware with `qemu` first.

```qemu-system-arm -M mps2-an386 -nographic -serial pty -semihosting -kernel elf/crypto_kem_bikel1_mf4_fi.elf```

This creates a pseudo terminal, bound to a file, which can be used as serial interface. This is usually located in `/dev/pts/`, the exact name is prompted and the variable `i` in the next block has to be set accordingly.
If `i` is set the next block can be used to setup the Communication_Target object to comfortably interact with the target.

In [ ]:
!cd ../pqm4 && make -j4 PLATFORM=mps2-an386

In [ ]:
lvl = 'l1'
i = ?

target = serial.Serial(f"/dev/pts/{i}", 38400)
t_com = com.Communication_Target(target, lvl)

In [ ]:
# choose a sub set of keys depending on the externel offset {eo}
tmp = list()
eo = 10

for e in key_dump:
    if e[0][2] == eo: tmp.append(e)
key_dump = tmp

In [ ]:
# compute DFR for keys
runs = 1000

target.read_all()

dfr = list()
print(len(key_dump))
for i,key in enumerate(key_dump):
    print(f"start with key {i+1}")
    t_com.w_sk(key[1].mupq_key)
    t_com.w_pk(key[1].pk)
    df = 0
    for _ in range(runs):
        t_com.encaps()	
        t_com.decaps()

        if not t_com.c_ss() == b"\x00":
            df = df + 1
    try:
        dfr.append(df/(runs))
    except ZeroDivisionError: continue
print(dfr)


In [ ]:
# save faulted keys
import shelve
with shelve.open("../traces/key_dump", writeback = True) as data:
	if ff in data.keys():
		data[ff].extend(key_dump[20:])
	else: data[ff] = key_dump
key_dump = list()

In [ ]:
# export into file
with open("/home/till/faulted_keys.txt", 'a') as file:
    for k in key_dump[:]:
        key = k[1]
        file.write(f"{key.mupq_key.hex()}\n")